# Generate Technical Indicators
---

### Import Libraries

In [1]:
import os
import requests 
import pandas_datareader as pdr

import functools
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns
import warnings

from sklearn import preprocessing 

from importlib import reload
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227

### Import Data

In [2]:
ticker = 'fb'

In [3]:
stock_df = pdr.get_data_yahoo(ticker, '1980')
# stock_df.to_csv('data/'+csv_title+'.csv')
stock_df.to_csv('data/raw_stocks/'+ticker.upper()+'.csv')

In [4]:
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-04-01,227.279999,222.699997,224.550003,224.850006,19533300,224.850006
2022-04-04,234.350006,225.860001,225.979996,233.889999,28054800,233.889999
2022-04-05,236.860001,231.020004,234.080002,231.839996,29727200,231.839996
2022-04-06,228.990005,221.410004,226.910004,223.300003,28969100,223.300003
2022-04-07,225.520004,220.041306,223.929993,220.070007,7244458,220.070007


In [5]:
stock_df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,2489.000000,2489.000000,2489.000000,2489.000000,2.489000e+03,2489.000000
mean,149.181800,145.614697,147.406476,147.442607,3.088831e+07,147.442607
std,90.753999,88.585065,89.647539,89.686274,2.745621e+07,89.686274
min,18.270000,17.549999,18.080000,17.730000,5.913100e+06,17.730000
25%,76.910004,75.080002,76.099998,76.080002,1.581160e+07,76.080002
50%,141.029999,139.009995,139.889999,139.940002,2.246520e+07,139.940002
75%,195.059998,190.220001,192.880005,192.940002,3.589900e+07,192.940002
max,384.329987,378.809998,381.679993,382.179993,5.735764e+08,382.179993


### Check for missing data

In [6]:
print('No Missing Data') if sum(stock_df.isna().sum())==0 else stock_df.isna().sum()

No Missing Data


---
# Technical Indicator Calculations  
---

### Momentum

In [7]:
def momentum(close, days):
    m = [None for i in range(days)]
    for i in range(len(close) - days):
        end = i + days
        m.append(close[i] - days)
    return m 

### Relative Strength Index (RSI)

In [8]:
def gain(x):
    if x > 0:
        return x
    else:
        return 0 
        
def loss(x):
    if x < 0:
        return abs(x)
    else:
        return 0 

    
def rsi(stock):
    rsi_list = [None for i in range(14)]
    stock = stock.Change
    
    # Calculate Relative Strength (RS) for first 15 
    avg_gain = sum([i for i in stock[1:15] if i > 0])/14
    avg_loss = sum([abs(i) for i in stock[1:15] if i < 0])/14 
    rs = avg_gain/avg_loss
    # Calculate Relative Strength Index (RSI) for first 15
    rsi = 100 - (100/(1+rs))
    rsi_list.append(rsi)
    
    # Calculate Relative Strength(RS) for 15+ 
    for i in range(15, len(stock)):
        avg_gain = (avg_gain * 13 + gain(stock[i]))/14
        avg_loss = (avg_loss * 13 + loss(stock[i]))/14
        rs = avg_gain/avg_loss
        
        # Calculate Relative Strength Index (RSI) for 15+ 
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    
    return rsi_list

### Moving Average Convergence/Divergence (MACD)

In [9]:
def macd(stock):
    exp1 = stock.Close.ewm(span=12, adjust=False).mean()
    exp2 = stock.Close.ewm(span=26, adjust=False).mean()
    ma_con_div = exp1-exp2
    signal = ma_con_div.ewm(span=9, adjust=False).mean()
    return ma_con_div, signal

### Bollinger Bands

In [10]:
def bollinger(stock, window=21):
    roll_mean = stock.Close.rolling(window).mean()
    roll_std = stock.Close.rolling(window).std()
    upper_band = roll_mean + (roll_std*2)
    lower_band = roll_mean - (roll_std*2)
    return upper_band, lower_band

---
# Generating Technical Indicators 
---

In [11]:
# gets list of all files and directories
# git add .ipynb_checkpoints directory -> causes issues 
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    if file.endswith('.csv'):
        name = file.lower().split(".")[0]
        stocks[name] = pd.read_csv('data/raw_stocks/'+file)
    
        # Date Feature
        stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
        stocks[name].set_index('Date', inplace = True)
        # Return Feature 
        stocks[name]['Return'] = round(stocks[name]['Close']/stocks[name]['Open']-1, 3)
        # Change Feature: Change previous day's price by absolute value
        stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
        # Volatility Feature
        stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
        # Moving Average: 7 Days
        stocks[name]['MA7'] = stocks[name].Close.rolling(window=7).mean()
        # Moving Average: 21 Days
        stocks[name]['MA21'] = stocks[name].Close.rolling(window=21).mean()
    
        # Momentum
        stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
        # Relative Strength Index (RSI)
        stocks[name]['RSI'] = rsi(stocks[name])
        # Moving Average Convergence/Divergence (MACD)
        stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
        # Bollinger Bands
        stocks[name]['Upper_Band'], stocks[name]['Lower_Band'] = bollinger(stocks[name])
    
        # Saving
        # stocks[name].dropna(inplace=True)
        stocks[name].to_csv('data/stocks/'+name+'.csv')

In [14]:
stocks[ticker.lower()].tail()

,High,Low,Open,Close,Volume,Adj Close,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_Band,Lower_Band
Date,,,,,,,,,,,,,,,,,
2022-04-01,227.279999,222.699997,224.550003,224.850006,19533300,224.850006,0.001,2.490005,42.202087,224.271432,208.440954,226.860001,54.617810,0.805737,-3.418365,237.812118,179.069790
2022-04-04,234.350006,225.860001,225.979996,233.889999,28054800,233.889999,0.035,9.039993,41.245671,226.317145,210.051907,224.850006,60.062217,1.924640,-2.349764,241.152522,178.951292
2022-04-05,236.860001,231.020004,234.080002,231.839996,29727200,231.839996,-0.010,-2.050003,40.301674,227.748572,212.164764,219.360001,58.352609,2.615808,-1.356650,242.847894,181.481634
2022-04-06,228.990005,221.410004,226.910004,223.300003,28969100,223.300003,-0.016,-8.539993,39.394519,227.707145,213.736669,221.850006,51.744907,2.446258,-0.596068,243.065398,184.407940
2022-04-07,225.520004,220.041306,223.929993,220.070007,7244458,220.070007,-0.017,-3.229996,38.533795,226.308574,214.763812,230.889999,49.463493,2.027879,-0.071279,243.352870,186.174754


In [22]:
new_df = pd.read_csv('data/stocks/'+ticker.lower()+'.csv')
new_df.isna().sum()

Date           0
High           0
Low            0
Open           0
Close          0
Volume         0
Adj Close      0
Return         0
Change         0
Volatility     1
MA7            6
MA21          20
Momentum       3
RSI           14
MACD           0
Signal         0
Upper_Band    20
Lower_Band    20
dtype: int64